In [1]:
url = 'https://raw.githubusercontent.com/chiarorosa/ia_aprendizado_maquina_basico/main/ml-dataset/kaggle-basico/diabetes.csv'

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Avaliação de Acurácia
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


In [2]:
df = pd.read_csv(url)
df.head()
X = df.drop('Outcome', axis=1)
y = df['Outcome']



In [3]:


# Dados de exemplo
df = np.random.normal(loc=0, scale=1, size=1000)

# Criar DataFrame
df = pd.DataFrame(df, columns=['Value'])

# Calcular quartis
q1 = df['Value'].quantile(0.25)
q3 = df['Value'].quantile(0.75)

# Calcular IQR
iqr = q3 - q1

# Definir limites para outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Remover outliers
df_clean = df[~((df < lower_bound) | (df > upper_bound)).any(axis=1)]

# Verificar o tamanho do DataFrame após a remoção dos outliers
print("Tamanho do DataFrame após a remoção dos outliers:", df_clean.shape[0])


Tamanho do DataFrame após a remoção dos outliers: 996


In [4]:

scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)


# Dividir em dois conjuntos, treinamento e teste, usando pareto 80/20
X_train, X_test, y_train, y_test = train_test_split(
    x_scaled,
    y,
    test_size=0.2, # pareto
    random_state=42 # reproduzivel
)

In [12]:
grade = {
    'alpha': np.linspace(0.0001, 0.001, 100),
    'fit_intercept': [True, False],
    'max_iter': np.arange(1000, 10000, 100),
    'tol': np.logspace(-6, -1, 6),  # Tolerância para critério de parada
}

Perception = RandomizedSearchCV(
    Perceptron(),
    grade,
    cv=5,
    scoring='accuracy',
    n_iter=20,  # Número de iterações de busca aleatória
    n_jobs=-1,  # Use todos os núcleos disponíveis
    random_state=42  # Para reprodutibilidade
)

Perception.fit(X_train, y_train)

melhor = Perception.best_estimator_

y_pred = melhor.predict(X_test)

acuracia = accuracy_score(y_test, y_pred)

print(f'Acurácia: {acuracia * 100:.2f}')
print(f'Melhor modelo: {melhor}')
print(f'Parâmetros do melhor modelo: {Perception.best_params_}')

guardaAcc = 0

with open('Melhor Acurracia Perception.txt', 'r+', encoding='utf-8') as f:
        try:
            linha = f.readline().strip()
            if linha.startswith('Acurácia:'):
                guardaAcc = float(linha.split(': ')[1]) / 100
                print(f'Acurácia anterior: {guardaAcc * 100:.2f}')
                if acuracia > guardaAcc:
                    f.seek(0) # vai pro inicio
                    f.write(f'Acurácia: {acuracia * 100:.2f} \n')
                    
                    f.truncate() #  remove residuos antigos
                    print(f'Acurácia atualizada: {acuracia * 100:.2f}')
                else:
                    print('Acurácia não atualizada')
        except Exception as e:
            print(f"Erro ao ler o arquivo: {e}")
       


Acurácia: 65.58
Melhor modelo: Perceptron(alpha=0.0009727272727272728, max_iter=1200, tol=0.0001)
Parâmetros do melhor modelo: {'tol': 0.0001, 'max_iter': 1200, 'fit_intercept': True, 'alpha': 0.0009727272727272728}
Acurácia anterior: 66.23
Acurácia não atualizada
